In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd 
%matplotlib inline
sns.set(style='darkgrid')
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import median_absolute_error as MAE
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor as XGBR
from sklearn.model_selection import GridSearchCV

In [2]:
df1=pd.read_csv('nashville_20190827200234.csv', parse_dates = ['Most Recent Sale Date', 'Sale Date'], dtype={'Zone': str, 'Neighborhood': str})

/home/drew/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Correcting badly-entered data
df1.loc[176025, 'Sale Price'] = 37000
df1.loc[27056, 'Sale Price'] = 161000
df1.loc[191004, 'Sale Price'] = 810000
df1.loc[239278, 'Sale Price'] = 280395
df1.loc[138891, 'Sale Price'] = 200000
df1.loc[241961, 'Sale Date'] = '2019-03-04'
df1.loc[241961, 'Sale Date'] = pd.to_datetime(df1.loc[241961, 'Sale Date'])
df1.loc[241961, 'Sale Price'] = 370000
df1.loc[230115, 'Sale Price'] = 325000
df1.loc[53151, 'Sale Price'] = 310900
df1.loc[259815, 'Sale Price'] = 1513142
df1.loc[129682, 'Sale Price'] = 1300000
df1.loc[154271, 'Square Footage Improved'] = 10094
df1.drop(190142, inplace=True) #Has a massive ADU ~4 times the size of the 'main' structure. Main struct is low-grade, ADU is luxury.
df1.drop(128094, inplace=True) #Can't find out anything about this house. Improbably large sqft and acreage, low price.

In [4]:
#Creating new dataframe without duplicate entires
df=df1.sort_values(by='Sale Date').drop_duplicates(subset='Map & Parcel', keep = 'last')
#Want to create a new column which is the mean sale price per square foot of a parcel in that neighborhood
df['PPS']=df['Sale Price']/df['Square Footage Improved']
df['PPS']=df['PPS'].replace(np.inf, np.nan)
meanpps=df.groupby('Neighborhood')['PPS'].mean().to_frame().rename(columns={'PPS':'NeighborhoodPPS'})
df=df.merge(meanpps, how='left', left_on = 'Neighborhood',right_index=True)
#Dropping parcels that were involved in multi-parcel sales
df=df[df['Multiple Parcels Involved in Sale'] == 'No']
#Testing the averaged assessment ratio idea.
df['Assessment Ratio'] = df['Assessment Land Improved'] / df['Total Appraisal Value Improved']
nbhdratio=df.groupby('Neighborhood')['Assessment Ratio'].mean().to_frame().rename(columns={'Assessment Ratio':'Nbhd Ratio'})
df=df.merge(nbhdratio, how='left', left_on = 'Neighborhood',right_index=True)
df['Month']= df['Sale Date'].dt.month
df['Quarter'] = df['Sale Date'].dt.quarter
df['Year'] = df['Sale Date'].dt.year
#Dropping some obscure categories we can't decipher
df['Building Grade']= df['Building Grade'].str.replace(r'\w\w\w', '').dropna()
#Dropping some outliers
df=df[df.PPS <= 1150]
df=df[df.Fixtures <=23]
df=df[df['Land Area Acres'] <=10]

In [5]:
TYPEDICT = {'HIGHRISE APT': 'HRISE', 'APARTMENT': 'CONDO', 'HRISE CONDO': 'HRISE',
            'RESD CONDO': 'CONDO', 'RW SING FAM': 'SINGLE FAM', 'SINGLE FAM': 'SINGLE FAM',
            'RZ SING FAM': 'SINGLE FAM', 'RH SING FAM': 'SINGLE FAM',
            'RY SING FAM': 'SINGLE FAM', 'MODULAR HOME': 'SINGLE FAM',
            'SING FAM': 'SINGLE FAM', 'RESD TRIPLEX': 'PLEX', 'RESD QUADPLX': 'PLEX',
            'R1 DUPLEX': 'PLEX', 'RES DUPLEX': 'PLEX', 'LODGE/FRAT\'L': 'THIRD',
            'CHURCH': 'THIRD', 'HEALTH CLUB': 'COMM', 'GROCERY/SMKT': 'COMM',
            'RETAIL/SHPG': 'COMM', 'DAYCARE': 'COMM', 'MARKET': 'COMM',
            'REST/BAR': 'COMM', 'OFFICE': 'OFFICE', 'WAREHOUSE': 'INDUSTRIAL',
            'EQUIP SHED': 'INDUSTRIAL', 'SERV GARAGE': 'INDUSTRIAL', 'TRUCK TERM': 'INDUSTRIAL',
            'ZERO LOT': 'VACANT', 'CAR WASH': 'COMM', 'MED OFC': 'OFFICE',
            'LAUNDRY': 'COMM', 'AUDITORIUM': 'THIRD', 'BANK': 'COMM', 'BOWLING': 'COMM',
            'BUSINESS CTR': 'COMM', 'CINEMA': 'COMM', 'CLUB/UNION': "THIRD",
            "COMM GRNHSE": "INDUSTRIAL", 'WHSE RETAIL': 'COMM', 'ENG/RESEARCH': 'OFFICE',
            'COUNTRY CLUB': 'THIRD', 'DEPT STORE': 'COMM', 'TENNIS': 'THIRD',
            'WALKUP APT': 'CONDO', 'DRUG STORE': 'COMM', 'DISC STORE': 'COMM',
            'ELDERLY HSG': 'CONDO', "FUNERAL HM": "THIRD", 'SALES SHOWRM': 'COMM',
            'LUMBER': 'INDUSTRIAL', 'MANF PLANT': 'INDUSTRIAL', 'MAINT HANGER': 'INDUSTRIAL',
            'MOTEL': 'COMM', 'MINI-LUBE': 'INDUSTRIAL', 'MINI-WAREHSE': 'INDUSTRIAL',
            'FAST FOOD': 'COMM', 'HOTEL': 'COMM', 'NURSING HOME': 'CONDO', "PARK'G GAR": 'COMM',
            'SERV STATION': 'COMM', 'INDOOR WP': 'COMM', 'HOSPITAL': 'THIRD', 'T-HANGER': 'INDUSTRIAL'
            }

df['Building Type Custom']=df['Building Type'].map(lambda x: TYPEDICT[x] if (x != '' and not pd.isnull(x)) else None)

In [6]:
df.PPS=df.PPS.replace(np.inf, np.nan)
df['Log Assessment']= df['Assessment Land Improved'].map(lambda x: np.log(x) if (pd.notnull(x) and x != 0) else None)
df['Log Assessment']=df['Log Assessment'].replace(np.inf, np.nan)
df['Log Fixtures']= df['Fixtures'].map(lambda x: np.log(x) if (pd.notnull(x) and x != 0) else None)
df['Log Fixtures']=df['Log Fixtures'].replace(np.inf, np.nan)
df['Log SqFt']= df['Square Footage Improved'].map(lambda x: np.log(x) if (pd.notnull(x) and x != 0) else None)
df['Log SqFt']=df['Log SqFt'].replace(np.inf, np.nan)
df['Log PPS']= df['PPS'].map(lambda x: np.log(x) if (pd.notnull(x) and x != 0) else None)
df['Log PPS']=df['Log PPS'].replace(np.inf, np.nan)
df['Log Land']= df['Land Area Acres'].map(lambda x: np.log(x) if (pd.notnull(x) and x != 0) else None)
df.Quarter= df.Quarter.map(lambda x: str(x) if pd.notnull(x) else '')
df.Year= df.Year.map(lambda x: str(x) if pd.notnull(x) else '')

In [13]:
df2 = df[['Exterior Wall','Log SqFt','Log Fixtures','Building Type Custom','Building Grade','Year','Log Assessment','NeighborhoodPPS', 'PPS', 'Quarter']].dropna()
X = pd.get_dummies(df2[['Exterior Wall','Year','NeighborhoodPPS','Log Assessment','Log Fixtures','Building Grade','Building Type Custom','Log SqFt', 'Quarter']])
y =df2['PPS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.28, random_state=42)

In [19]:
import statsmodels as sm
from statsmodels.regression.linear_model import GLS

In [27]:
y_test= list(y_test)
model = GLS(y_train, X_train)
results=model.fit()

In [28]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                    PPS   R-squared:                       0.739
Model:                            GLS   Adj. R-squared:                  0.739
Method:                 Least Squares   F-statistic:                     3785.
Date:                Thu, 29 Aug 2019   Prob (F-statistic):               0.00
Time:                        21:57:40   Log-Likelihood:            -2.0046e+05
No. Observations:               38812   AIC:                         4.010e+05
Df Residuals:                   38782   BIC:                         4.012e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
NeighborhoodPPS                     0.0414      0.001     27.945      0.000       0.038       0.044
Log Assessment                     74.0379      0.400    184.960      0.000      73.253      74.822
Log Fixtures                       31.4955      1.134     27.777      0.000      29.273      33.718
Log SqFt                         -112.4709      1.048   -107.285      0.000    -114.526    -110.416
Exterior Wall_BRICK                38.1073      1.939     19.656      0.000      34.307      41.907
Exterior Wall_BRICK/FRAME          32.0565      1.958     16.370      0.000      28.218      35.895
Exterior Wall_CONC BLK             35.4917      4.469      7.942      0.000      26.733      44.251
Exterior Wall_FRAME                32.3778      1.909     16.958      0.000      28.636      36.120
Exterior Wall_FRAME/STONE          28.5450      3.640      7.843      0.000      21.411      35.679
Exterior Wall_GLASS                96.9604      3.486     27.811      0.000      90.127     103.794
Exterior Wall_LOG                  68.9002     12.318      5.594      0.000      44.757      93.044
Exterior Wall_METAL              -171.8577     11.389    -15.090      0.000    -194.180    -149.536
Exterior Wall_PRECAST CONC      -1.693e-12   4.96e-13     -3.417      0.001   -2.66e-12   -7.22e-13
Exterior Wall_STONE                45.5808      2.948     15.460      0.000      39.802      51.360
Exterior Wall_STUCCO               43.1437      2.407     17.922      0.000      38.425      47.862
Year_2015                          20.4101      1.542     13.239      0.000      17.388      23.432
Year_2016                          36.8251      1.535     23.993      0.000      33.817      39.833
Year_2017                          51.9699      1.531     33.939      0.000      48.969      54.971
Year_2018                          65.3753      1.523     42.917      0.000      62.390      68.361
Year_2019                          74.7253      1.622     46.071      0.000      71.546      77.904
Building Grade_A                   86.5852      2.056     42.106      0.000      82.555      90.616
Building Grade_B                   52.0044      1.590     32.716      0.000      48.889      55.120
Building Grade_C                   35.7239      1.397     25.571      0.000      32.986      38.462
Building Grade_D                    8.0371      1.664      4.831      0.000       4.776      11.298
Building Grade_E                  -25.1007      5.279     -4.754      0.000     -35.448     -14.753
Building Grade_X                   92.0558      2.531     36.376      0.000      87.096      97.016
Building Type Custom_CONDO         40.4340      1.563     25.869      0.000      37.370      43.498
Building Type Custom_HRISE        197.4783      2.453     80.503      0.000     192